# PyCity Schools Analysis

- In this provided analysis, we have collected multiple sets of data to compare schools, grade level, budget, count of students, math and reading scores, as well as other items. 

- Comparisons were brought to light while sifting this data. One camparison brought to light is the higher scoring student who attend charter schools versus district schools. Referring to the top 5 and bottom 5 performing schools, Higher rated schools were all charter and lower rated schools were all district.
- While thinking budget per student could play a role on education value, another trend recognized was that spending per student is not a factor for higher grades. The opposite actually occured where students where spending was lower, the average overall passing % was higher than all else.
  
---

In [23]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [26]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["school_name"].unique())
school_count

15

In [25]:
# Calculate the total number of students
student_count = school_data_complete['Student ID'].nunique()
student_count

39170

In [27]:
# Calculate the total budget
total_budget = school_data_complete["budget"].unique().sum()
total_budget

24649428

In [28]:
# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean()
average_math_score

78.98537145774827

In [29]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean()
average_reading_score

81.87784018381414

In [30]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [32]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [35]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [36]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary =  pd.DataFrame({'Total Schools': [school_count], 
                                  'Total Students': [student_count], 
                                  'Total Budget':[total_budget],
                                   'Average Math Score':[average_math_score],
                                   'Average Reading Score':[average_reading_score],
                                   '% Passing Math':[passing_math_percentage],
                                   '% Passing Reading':[passing_reading_percentage],
                                   '% Overall Passing': [overall_passing_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [38]:
# Use the code provided to select all of the school types
school_types = school_type = school_data.set_index("school_name")["type"]

In [47]:
# Calculate the total student count per school
per_school_counts = school_data_complete.groupby("school_name").size()

In [48]:
# Calculate the total school budget and per capita spending per school
per_school_budget = school_data.set_index("school_name")["budget"]
per_school_capita = per_school_budget / per_school_counts

In [59]:
# Calculate the average test scores per school
per_school_math = school_data_complete.set_index(["school_name"]).groupby("school_name")["math_score"].mean()
per_school_reading = school_data_complete.set_index(["school_name"]).groupby("school_name")["reading_score"].mean()

In [60]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].count()["student_name"]
school_students_passing_math = students_passing_math / float(student_count) * 100

In [61]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].count()["student_name"]
school_students_passing_reading = students_passing_reading / float(student_count) * 100

In [62]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [57]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

In [64]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({"School Type": school_types,
                                   "Total Students": per_school_counts,
                                   "Total School Budget": per_school_budget,
                                   "Per Student Budget": per_school_capita,
                                   "Average Math Score": per_school_math,
                                   "Average Reading Score": per_school_reading,
                                   "% Passing Math": per_school_passing_math,
                                   "% Passing Reading": per_school_passing_reading,
                                   "% Overall Passing": overall_passing_rate})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,1.506850,1.724386,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,4.035568,4.618163,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,2.542586,2.909646,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,2.737527,3.132730,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,5.107688,5.845059,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,1.617710,1.851251,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,17.559919,20.094956,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,2.570478,2.941565,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,1.574897,1.802257,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [65]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,4.035568,4.618163,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,4.585985,5.248041,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,5.107688,5.845059,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,3.284312,3.758452,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,7.794267,8.919487,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [67]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(["% Overall Passing"])
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,1.874990,2.145673,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,2.542586,2.909646,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,2.570478,2.941565,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,1.617710,1.851251,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,1.574897,1.802257,53.539172


## Math Scores by Grade

In [76]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" 
ninth_graders_scores = ninth_graders.groupby(['school_name']).mean(numeric_only=True)
tenth_graders_scores = tenth_graders.groupby(['school_name']).mean(numeric_only=True)
eleventh_graders_scores = eleventh_graders.groupby(['school_name']).mean(numeric_only=True)
twelfth_graders_scores = twelfth_graders.groupby(['school_name']).mean(numeric_only=True)

# Use the code to select only the `math_score`.
ninth_grader_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame(
    {
        '9th Grade Average Math Score':ninth_grader_math_scores,
        '10th Grade Average Math Score':tenth_grader_math_scores,
        '11th Grade Average Math Score':eleventh_grader_math_scores,
        '12th Grade Average Math Score':twelfth_grader_math_scores
    }
)

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th Grade Average Math Score,10th Grade Average Math Score,11th Grade Average Math Score,12th Grade Average Math Score
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [78]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({'Ninth Grade': ninth_grade_reading_scores,
                                         'Tenth Grade': tenth_grader_reading_scores,
                                         'Eleventh Grade': eleventh_grader_reading_scores,
                                         'Twelfth Grade': twelfth_grader_reading_scores
                                        })

# Formatting, including rounding and index name removalreading_scores_by_grade.index.name = None
reading_scores_by_grade['Ninth Grade'] = reading_scores_by_grade['Ninth Grade'].map('{:.2f}'.format)
reading_scores_by_grade['Tenth Grade'] = reading_scores_by_grade['Tenth Grade'].map('{:.2f}'.format)
reading_scores_by_grade['Eleventh Grade'] = reading_scores_by_grade['Eleventh Grade'].map('{:.2f}'.format)
reading_scores_by_grade['Twelfth Grade'] = reading_scores_by_grade['Twelfth Grade'].map('{:.2f}'.format)

# Display the DataFrame
reading_scores_by_grade

,Ninth Grade,Tenth Grade,Eleventh Grade,Twelfth Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [87]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [88]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [89]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=labels)
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,1.506850,1.724386,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,4.035568,4.618163,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,2.542586,2.909646,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,2.737527,3.132730,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,5.107688,5.845059,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,1.617710,1.851251,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,17.559919,20.094956,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,2.570478,2.941565,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,1.574897,1.802257,53.539172,$645-680


In [91]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [93]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score" : spending_math_scores,
                    "Average Reading Score" : spending_reading_scores,
                    "% Passing Math" : spending_passing_math,
                    "% Passing Reading" : spending_passing_reading,
                    "% Overall Passing" : overall_passing_spending})

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,7.261350,8.309635,90.369459
$585-630,81.899826,83.155286,4.666665,5.340368,81.418596
$630-645,78.518855,81.624473,2.935272,3.359022,62.857656
$645-680,76.997210,81.027843,1.921028,2.198358,53.526855


## Scores by School Size

In [95]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [96]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_counts, size_bins, labels=labels)
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
school_name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,1.506850,1.724386,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,4.035568,4.618163,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,2.542586,2.909646,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,2.737527,3.132730,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,5.107688,5.845059,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,1.617710,1.851251,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,17.559919,20.094956,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,2.570478,2.941565,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,1.574897,1.802257,53.539172,Large (2000-5000)


In [97]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [99]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({"Average Math Score": size_math_scores,
                             "Average Reading Score": size_reading_scores,
                             "% Passing Math": size_passing_math,
                             "% Passing Reading": size_passing_reading,
                             "% Overall Passing": size_overall_passing})

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,12.677093,14.507222,89.883853
Medium (1000-2000),83.374684,83.864438,4.430540,5.070155,90.621535
Large (2000-5000),77.746417,81.344493,2.213669,2.533245,58.286003


## Scores by School Type

In [101]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [102]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({"Average Math Score" : average_math_score_by_type,
                            "Average Reading Score" : average_reading_score_by_type,
                            "% Passing Math" : average_percent_passing_math_by_type,
                            "% Passing Reading" : average_percent_passing_reading_by_type,
                            "% Overall Passing" : average_percent_overall_passing_by_type})
# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,6.34890,7.265459,90.432244
District,76.956733,80.966636,2.06072,2.358215,53.672208
